# DESC Based Sampling Demo
This notebook demonstrates how to sample an omnigenous field and targets, run a boundary optimization, and visualize the resulting field in Boozer coordinates.

In [ ]:
from constellaration.omnigeneity.omnigenity_field_sampling import (
    SampleOmnigenousFieldAndTargetsSettings,
)
from constellaration.data_generation import desc_optimization_settings
from constellaration.data_generation import desc_optimization
from constellaration.mhd import (
    vmec_utils,
    vmec_settings as vmec_settings_module,
    ideal_mhd_parameters as ideal_mhd_parameters_module,
)
from constellaration.boozer import boozer as boozer_module
from constellaration.utils import visualization
from constellaration.omnigeneity import omnigenity_field_plot

## 1. Sample an Omnigenous Field and Targets

In [ ]:
# Sample a single omnigenous field and targets
sampler = SampleOmnigenousFieldAndTargetsSettings(n_samples=1, seed=7)
sample = sampler.sample_omnigenous_fields_and_targets()[0]

# Inspect sample
print(sample.model_dump_json(indent=4))

In [ ]:
omnigenity_field_plot.plot_boozer_field(sample.omnigenous_field);

## 2. Run DESC Boundary Optimization

In [ ]:
# Run for ~3 mins
settings = desc_optimization_settings.DescOmnigenousFieldOptimizationSettings()
settings.optimizer_settings.maxiter = 50
optimized_surface = desc_optimization.optimize_boundary_omnigenity_desc(
    sample, settings
)

## 3. Plot the resulting boundary and its field in Boozer coordinates

In [ ]:
visualization.plot_boundary(optimized_surface)

In [ ]:
# Compute the equilibrium
vmec_preset_settings = vmec_settings_module.VmecPresetSettings(
    fidelity="low_fidelity",
)
vmec_settings = vmec_settings_module.create_vmec_settings_from_preset(
    optimized_surface,
    settings=vmec_preset_settings,
)
ideal_mhd_parameters = ideal_mhd_parameters_module.boundary_to_ideal_mhd_parameters(
    optimized_surface
)
equilibrium = vmec_utils.run_vmec(
    boundary=optimized_surface,
    mhd_parameters=ideal_mhd_parameters,
    vmec_settings=vmec_settings,
)

In [ ]:
# Visualize the target field
omnigenity_field_plot.plot_boozer_field(sample.omnigenous_field)

# Visualize the optimized field
boozer_settings = boozer_module.BoozerSettings(normalized_toroidal_flux=[1])
final_field_figure = visualization.plot_boozer_surfaces(
    equilibrium=equilibrium,
    settings=boozer_settings,
)[0]
final_field_figure.show()